# 🧑‍🍳 Preparation and Imports

In [ ]:
!pip install modelbit

In [ ]:
import pandas as pd
from prophet import Prophet
import modelbit
mb = modelbit.login()

In [ ]:
mb.switch_branch("harry_dev")

# ⚙️ Data Engineering

In [ ]:
#Set holidays for use in model
xmas = pd.DataFrame({
    'holiday': 'xmas',
    'ds': pd.to_datetime(['2019-12-31', '2020-12-31','2021-12-31', '2022-12-31','2023-12-31','2024-12-31']),
    'lower_window': -10,
    'upper_window': 0,
})

In [ ]:
orders = pd.read_csv(r'orders.csv', parse_dates=['ds'], names = ['ds','y'], header=0)

In [ ]:
orders_weekly = orders.groupby([pd.Grouper(key='ds', freq='W')])['y'].sum().reset_index()

In [ ]:
#Set week beginning to Sunday to link up with Quicksight. For Monday it is '6'
orders_weekly['ds'] -= pd.Timedelta(days=7)

In [ ]:
#Delete final two weeks of dataframe which will include incomplete data and dates after April 2020
orders_weekly = orders_weekly[:-2]
orders_weekly = orders_weekly[~(orders_weekly['ds'] < '2020-04-30')]

# 🧠 Model Fitting

In [ ]:
m = Prophet(holidays=xmas)
m.fit(orders_weekly)

# 📈 Model Evaluation

In [ ]:
from prophet.plot import plot_yearly
future = m.make_future_dataframe(periods=100, freq='W')
fcst = m.predict(future)
fig = m.plot(fcst)

# 🚀 Model Deployment

In [ ]:
def prophet_forecast(dates_df: pd.DataFrame) -> pd.DataFrame:
  fcst = m.predict(dates_df)
  fcst = fcst[["ds","yhat_lower", "yhat_upper","yhat"]]
  return fcst

prophet_forecast(m.make_future_dataframe(periods=100, freq='W'))

In [ ]:
mb.deploy(prophet_forecast,
          dataframe_mode=True,
          example_dataframe=m.make_future_dataframe(periods=100, freq='W'))